In [1]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *
from src.validation import validate_jumpstart_cube, display_validate_results
from src.coherence import analyze_deck_theme_coherence_enhanced

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
cube_df = pd.read_csv('JumpstartCube_ThePauperCube_ULTIMATE_Final_v3.csv')

In [2]:
display_validate_results(validate_jumpstart_cube(cube_df, oracle_df))

### Cube is validass! 🎉

#### Deck Summaries:

**Azorius Evasion/Flying** 13 cards

**Black Aggro** 13 cards

**Black Control** 13 cards

...

**Simic Control** 13 cards

**White Aggro** 13 cards

**White Control** 13 cards

**White Equipment** 13 cards

**White Tokens** 13 cards

In [3]:
analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)["Green Big Creatures"]['overall_coherence']

43.06892307692308

# Generated

In [ ]:
# Import the new swap recommendation function

# Get all deck coherence results first
from src.improve import display_swap_recommendations, find_best_card_swaps_for_deck


coherence_results = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)

# Find the deck with the lowest coherence score for improvement
worst_deck = min(coherence_results.items(), key=lambda x: x[1]['overall_coherence'])
deck_to_improve = worst_deck[0]

print(f"Deck with lowest coherence: {deck_to_improve} (Score: {worst_deck[1]['overall_coherence']:.1f})")

# Find the best card swaps for this deck
swap_recommendations = find_best_card_swaps_for_deck(
    deck_to_improve, 
    cube_df, 
    oracle_df, 
    coherence_results, 
    num_swaps=2
)

# Display the recommendations
display_swap_recommendations(swap_recommendations)

ModuleNotFoundError: No module named 'jumpstart'

In [ ]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck
specific_deck = "Green Big Creatures"

if specific_deck in coherence_results:
    print(f"Analyzing {specific_deck} for potential improvements...")
    
    # Get swap recommendations for this specific deck
    specific_swaps = find_best_card_swaps_for_deck(
        specific_deck, 
        cube_df, 
        oracle_df, 
        coherence_results, 
        num_swaps=2
    )
    
    # Display the recommendations
    display_swap_recommendations(specific_swaps)
else:
    print(f"Deck '{specific_deck}' not found. Available decks:")
    for deck in sorted(coherence_results.keys()):
        score = coherence_results[deck]['overall_coherence']
        print(f"  - {deck} (Score: {score:.1f})")

Analyzing Green Big Creatures for potential improvements...
Analyzing deck: Green Big Creatures
Current coherence: 43.1
Expected themes: Big Creatures
Deck colors: G
Found 27 candidate cards to consider
Identified 13 cards as potential removal candidates


# 🔄 Swap Recommendations for Green Big Creatures

**Current Coherence:** 43.1/100

**Expected Themes:** Big Creatures

## 🎯 Recommended Changes

...

### Cards to Add:

- **Bannerhide Krushok** (Theme Score: 4.0) - from Green Midrange

- **Bramble Wurm** (Theme Score: 4.0) - from Simic Control

---

💡 **Next Steps:** Use these recommendations to manually update your cube CSV file, then re-run the coherence analysis to confirm the improvements.

# 🔄 Card Swap Recommendation System

The new `find_best_card_swaps_for_deck()` function helps improve deck coherence by finding optimal card swaps. Here's how it works:

## 🎯 What it does:
1. **Analyzes current deck weaknesses** - Identifies cards that don't fit the theme well
2. **Finds potential replacements** - Looks at both unassigned cards from the oracle pool and cards from other decks
3. **Calculates theme scores** - Evaluates how well each card matches the deck's expected themes
4. **Recommends optimal swaps** - Suggests the best combination of cards to remove and add

## 📊 Key Features:
- **Theme-based scoring**: Uses your existing theme keywords to evaluate card fit
- **Color identity compliance**: Ensures recommendations respect mana requirements  
- **Cross-deck optimization**: Can suggest moving cards between decks for better overall coherence
- **Incremental improvement**: Focuses on meaningful improvements rather than drastic changes

## 🛠️ Parameters:
- `deck_name`: The specific deck to improve
- `num_swaps`: How many cards to swap (default: 2)
- `cube_df`: Your current jumpstart cube
- `oracle_df`: The full pauper cube as source pool
- `coherence_results`: Pre-calculated coherence analysis

## 📈 Usage Tips:
- Start with the lowest-scoring decks for maximum impact
- Run coherence analysis after implementing swaps to verify improvements
- Consider the "source" of recommended cards (unassigned vs. from other decks)
- Use this iteratively - make swaps, re-analyze, repeat